In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import fastf1
from fastf1 import plotting
from scipy.interpolate import interp1d

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025, 13, 'Qualifying')
session.load()

In [ ]:
pia_color = plotting.get_driver_color(session=session,identifier='PIA')
nor_color = 'yellow'

In [ ]:
nor_df = session.laps.pick_drivers('NOR').pick_fastest().telemetry[['Time','Distance','X','Y','Speed']].copy()
pia_df = session.laps.pick_drivers('PIA').pick_fastest().telemetry[['Time','Distance','X','Y','Speed']].copy()

In [ ]:
nor_df['Seconds'] = nor_df['Time'].dt.total_seconds()
pia_df['Seconds'] = pia_df['Time'].dt.total_seconds()

In [ ]:
# Norris
nor_time_interp = interp1d(nor_df['Distance'], nor_df['Seconds'], kind='linear', bounds_error=False, fill_value="extrapolate")
nor_x_interp    = interp1d(nor_df['Distance'], nor_df['X'], kind='linear', bounds_error=False, fill_value="extrapolate")
nor_y_interp    = interp1d(nor_df['Distance'], nor_df['Y'], kind='linear', bounds_error=False, fill_value="extrapolate")

# Piastri
pia_time_interp = interp1d(pia_df['Distance'], pia_df['Seconds'], kind='linear', bounds_error=False, fill_value="extrapolate")
pia_x_interp    = interp1d(pia_df['Distance'], pia_df['X'], kind='linear', bounds_error=False, fill_value="extrapolate")
pia_y_interp    = interp1d(pia_df['Distance'], pia_df['Y'], kind='linear', bounds_error=False, fill_value="extrapolate")

In [ ]:
min_dist = max(pia_df['Distance'].min(), nor_df['Distance'].min())
max_dist = min(pia_df['Distance'].max(), nor_df['Distance'].max())

distance_grid = np.linspace(min_dist, max_dist, num=1000)

In [ ]:
# Interpolated cumulative time
nor_time = pd.Series(nor_time_interp(distance_grid)).diff().dropna()
pia_time = pd.Series(pia_time_interp(distance_grid)).diff().dropna()

# Interpolated positions
y = (nor_x_interp(distance_grid) + pia_x_interp(distance_grid)) / 2
x = -(nor_y_interp(distance_grid) + pia_y_interp(distance_grid)) / 2

In [ ]:
delta = pia_time - nor_time

# Assign colors
colors = np.where(delta < 0, pia_color, nor_color)

In [ ]:
# Create line segments
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
import matplotlib.patches as mpatches

# Create legend handles manually
pia_patch = mpatches.Patch(color=pia_color, label='Piastri faster')
nor_patch = mpatches.Patch(color=nor_color, label='Norris faster')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
fig.set_facecolor('#000000')
ax.axis('off')

# Color per segment (drop last color to match segments length)
lc = LineCollection(segments, colors=colors, linewidth=8)

ax.add_collection(lc)
ax.autoscale()

# Add legend to your axes
ax.legend(handles=[pia_patch, nor_patch],
          loc='upper left',  # or any location you prefer
          bbox_to_anchor=(0.04,1),
          frameon=True,
          facecolor='k',
          fontsize=20,
          labelcolor='white')

plt.tight_layout()
plt.show()